# Wstęp

Celem tego projektu jest przeprowadzenie klasyfikacji na wybranej bazie danych z wykorzystaniem podstawowych klasyfikatorów:
[_K najbliższych sąsiadów_](https://pl.wikipedia.org/wiki/K_najbli%C5%BCszych_s%C4%85siad%C3%B3w),
[_naiwny klasyfikator bayesowski_](https://pl.wikipedia.org/wiki/Naiwny_klasyfikator_bayesowski),
[_drzewo decyzyjne_](https://pl.wikipedia.org/wiki/Drzewo_decyzyjne) oraz
[_sieć neuronowa_](https://pl.wikipedia.org/wiki/Sie%C4%87_neuronowa).

## Omówienie wybranej bazy danych

Wybrana przeze mnie baza danych zawiera informacje o [diamentach](https://www.kaggle.com/datasets/shivam2503/diamonds).
Składa się ona z $53941$ wierszy i następujących $11$ kolumn:

1. **indeks**
2. **ilość karatów** (carat) - wartości od $0.2$ do $5.01$
3. **jakość cięcia** (quality of the cut) - wartości _Fair_, _Good_, _Very Good_, _Premium_, _Ideal_
4. **kolor** (color) - wartości od _J_ (najgorszy) do _D_ (najlepszy)
5. **klarowność** (clarity) - wartości _I1_ (najgorsza), _SI2_, _SI1_, _VS2_, _VS1_, _VVS2_, _VVS1_, _IF_ (najlepsza)
6. **dlugość** (x) - długość w milimetrach, wartości od $0$ do $10.74$
7. **szerokość** (y) - szerokość w milimetrach, wartości od $0$ do $58.9$
8. **głębokość** (z) - głębokość w milimetrach, wartości od $0$ do $31.8$
9. **łączny procent głębokości** (depth) - wyliczony ze wzoru $2z/(x + y)$, wartości od $43$ do $79$
10. **szerokość szczytu diamentu względem jego najszerszego punktu** (table) - wartości od $43$ do $95$
11. **cena** (price) - cena diamentu w dolarach, od $\$326$ do $\$18,823$

W mojej pracy będę starał się przewidzieć cenę diamentu na podstawie jego pozostałych cech.

# Przygotowanie danych

W celu przeprowadzenia eksperymentu, do klasyfikacji przygotuje 2 bazy danych o różnym stopniu modyfikacji.
Będą do tego potrzebne następujące biblioteki:

- Pandas
- Numpy

Poniżej zostają one zaimportowane, wczytuję także dane z pliku [_diamonds.csv_](diamonds.csv) oraz wyświetlam ich krótką charakterystykę:


In [52]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

diamonds = pd.read_csv("diamonds.csv", index_col=0)

diamonds.describe()


,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


W powyższej tabeli widać jak rozkładają się dane liczbowe zawarte w bazie danych.

## Przygotowanie mniej przetworzonej bazy danych

### Zamiana wartości tekstowych na liczby

Pierwszym ważnym krokiem jest zamiana wartości tekstowych na liczbowe, ponieważ wybrane przez nas algorytmy mogą działać tylko na danych liczbowych. Pierwszą z takich kolumn jest _jakość cięcia_ (cut). W celu zamiany tych wartości można skorzystać z _LabelEncoder_ z biblioteki _sklearn_:


In [53]:
LE = LabelEncoder()

first_database = diamonds.copy()

first_database.sort_values(by="cut", inplace=True)
first_database["cut"] = LE.fit_transform(first_database["cut"])


Wartości zostały zmienione w następujący sposób:

| Wartość przed zamianą | Wartość po zamianie |
| --------------------- | :-----------------: |
| Fair                  |          0          |
| Good                  |          1          |
| Very Good             |          2          |
| Premium               |          3          |
| Ideal                 |          4          |

Drugą kolumną, w której trzeba dokonać podobnych zmian, jest _kolor_ (color). Ponownie można skorzystać z _LabelEncoder_:


In [54]:
first_database.sort_values(by="color", inplace=True)
first_database["color"] = LE.fit_transform(first_database["color"])


Kolory zostały zmodyfikowane w następujący sposób:

| Wartość przed zamianą | Wartość po zamianie |
| --------------------- | :-----------------: |
| D                     |          0          |
| E                     |          1          |
| F                     |          2          |
| G                     |          3          |
| H                     |          4          |
| I                     |          5          |
| J                     |          6          |

Ostatnią tekstową kolumną jest _klarowność_ (clarity). Tym razem trzeba było skorzystać z ręcznie napisanej funkcji, ponieważ danych w tej kolumnie nie można było posortować zgodnie z wartościami, które reprezentują:


In [59]:
def change_clarity(value):
    res = 0
    match value:
        case "I1": res = 0
        case "SI2": res = 1
        case "SI1": res = 2
        case "VS2": res = 3
        case "VS1": res = 4
        case "VVS2": res = 5
        case "VVS1": res = 6
        case "IF": res = 7
        case _: res = value

    return res


first_database["clarity"] = first_database["clarity"].apply(change_clarity)


Klarowność została zmieniona w następujący sposób:

| Wartość przed zamianą | Wartość po zamianie |
| --------------------- | :-----------------: |
| I1                    |          0          |
| SI2                   |          1          |
| SI1                   |          2          |
| VS2                   |          3          |
| VS1                   |          4          |
| VVS2                  |          5          |
| VVS1                  |          6          |
| IF                    |          7          |

Po tym etapie modyfikacji baza danych wygląda w następujący sposób:


In [60]:
first_database.describe()


,carat,cut,color,clarity,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,2.553003,2.594197,3.051020,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.027708,1.701105,1.647136,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,0.000000,0.000000,0.000000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,2.000000,1.000000,2.000000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,2.000000,3.000000,3.000000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,3.000000,4.000000,4.000000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,4.000000,6.000000,7.000000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000
